In [ ]:
%%capture
! sh pull.sh

In [ ]:
cd moscontest

In [ ]:
%%capture
! dvc pull data/processed/processed.zip.dvc
! unzip 'data/processed/processed.zip'

In [1]:
import torch
import torchvision
from catalyst import utils, dl
from src.nn import FERFC, AUCCallback, PrecisionRecallF1ScoreCallback, ConfusionMatrixCallback
from src.data.features import BIOMETR
utils.prepare_cudnn(deterministic=True)
utils.set_global_seed(7)
device = utils.get_device()

In [2]:
kwargs = {'batch_size': 1000, 'num_workers': 8}
threshold = 10000
dataset = BIOMETR(exclude=['contempt'])
train_dataloader = dataset.data_loader(end=threshold, shuffle=True, drop_last=True, **kwargs)
valid_dataloader = dataset.data_loader(start=threshold, **kwargs)
loaders = {'train': train_dataloader, 'valid': valid_dataloader}

In [3]:
class_names = dataset.classes
num_classes = len(class_names)
model = FERFC(num_classes)
weight = dataset._weight_classes().to(device)
criterion = torch.nn.CrossEntropyLoss(weight=weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1)
callbacks = [
    AUCCallback(num_classes=num_classes, class_names=class_names),
    PrecisionRecallF1ScoreCallback(num_classes=num_classes, class_names=class_names),
    ConfusionMatrixCallback(num_classes=num_classes, class_names=class_names),
]
logdir = 'logs/FERFC/'

In [4]:
weight

tensor([1.1850, 1.0373, 3.0398, 0.1858, 0.4024, 0.4931, 0.6567],
       device='cuda:0')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $logdir

In [ ]:
runner = dl.SupervisedRunner(device=device)
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    callbacks=callbacks,
    loaders=loaders,
    logdir=logdir,
    num_epochs=200,
    verbose=True,
)

In [ ]:
! sh ../push.sh $logdir